In [1]:
import pandas as pd
import numpy as np

In [3]:
df = pd.read_csv('/content/diabetes.csv')
df.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [4]:
df.corr()['Outcome']

,Outcome
Pregnancies,0.221898
Glucose,0.466581
BloodPressure,0.065068
SkinThickness,0.074752
Insulin,0.130548
BMI,0.292695
DiabetesPedigreeFunction,0.173844
Age,0.238356
Outcome,1.000000


In [5]:
X = df.iloc[:,:-1].values
y = df.iloc[:,-1].values

In [7]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

In [8]:
X = scaler.fit_transform(X)

In [9]:
X.shape

(768, 8)

In [10]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=1)

In [105]:
import tensorflow
from tensorflow import keras
from keras import Sequential
from keras.layers import Dense,Dropout

In [28]:
model = Sequential()
model.add(Dense(32,activation='relu',input_dim=8))
model.add(Dense(1,activation='sigmoid'))
model.compile(optimizer='Adam',loss='binary_crossentropy',metrics=['accuracy'])

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [29]:
model.fit(X_train,y_train,batch_size = 32 , epochs=100,validation_data=(X_test,y_test))

Epoch 1/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 1s 16ms/step - accuracy: 0.4945 - loss: 0.7254 - val_accuracy: 0.5649 - val_loss: 0.6676
Epoch 2/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.6580 - loss: 0.6538 - val_accuracy: 0.7208 - val_loss: 0.6199
Epoch 3/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.6841 - loss: 0.6250 - val_accuracy: 0.7532 - val_loss: 0.5857
Epoch 4/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7251 - loss: 0.5737 - val_accuracy: 0.7792 - val_loss: 0.5575
Epoch 5/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7446 - loss: 0.5395 - val_accuracy: 0.7857 - val_loss: 0.5370
Epoch 6/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7481 - loss: 0.5337 - val_accuracy: 0.7922 - val_loss: 0.5223
Epoch 7/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7483 - loss: 0.5140 - val_accuracy: 0.8117 - val_loss: 0.5109
Epoch 8/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.7788 - loss: 0.4960 - val_accuracy: 0.7987 - 

In [21]:
!pip install keras-tuner
import keras_tuner as kt

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.1/129.1 kB 2.5 MB/s eta 0:00:00


#**1.How to select appropriate optimizer**
# **2.No, Of nodes in a layer**
# **3.How to select no.of layers**
# **4.All in all one Model**

In [30]:
def build_model(hp):
  model = Sequential()
  model.add(Dense(32,activation='relu',input_dim=8))
  model.add(Dense(1,activation='sigmoid'))
  optimizer=hp.Choice('optimizer',values = ['Adam','sgd','rmsprop','adadelta'])
  model.compile(optimizer=optimizer,loss='binary_crossentropy',metrics=['accuracy'])
  return model

In [31]:
tuner = kt.RandomSearch(build_model,objective='val_accuracy',max_trials=5)

Reloading Tuner from ./untitled_project/tuner0.json


In [32]:
tuner.search(X_train,y_train,epochs=5,validation_data=(X_test,y_test))

Trial 4 Complete [00h 00m 03s]
val_accuracy: 0.7922077775001526

Best val_accuracy So Far: 0.7922077775001526
Total elapsed time: 00h 02m 30s


In [33]:
tuner.get_best_hyperparameters()[0].values

{'optimizer': 'sgd'}

In [34]:
model = tuner.get_best_models(num_models=1)[0]

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [35]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 32)             │           288 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 321 (1.25 KB)

 Trainable params: 321 (1.25 KB)

 Non-trainable params: 0 (0.00 B)

In [37]:
model.fit(X_train,y_train,batch_size = 32 , epochs=100,initial_epoch=6,validation_data=(X_test,y_test))

Epoch 7/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - accuracy: 0.7577 - loss: 0.5629 - val_accuracy: 0.8052 - val_loss: 0.5406
Epoch 8/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.7750 - loss: 0.5515 - val_accuracy: 0.7987 - val_loss: 0.5336
Epoch 9/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.7506 - loss: 0.5467 - val_accuracy: 0.7987 - val_loss: 0.5276
Epoch 10/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7689 - loss: 0.5339 - val_accuracy: 0.7987 - val_loss: 0.5227
Epoch 11/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7523 - loss: 0.5310 - val_accuracy: 0.8052 - val_loss: 0.5180
Epoch 12/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7346 - loss: 0.5395 - val_accuracy: 0.8052 - val_loss: 0.5138
Epoch 13/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7480 - loss: 0.5189 - val_accuracy: 0.7987 - val_loss: 0.5099
Epoch 14/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7459 - loss: 0.5202 - val_accuracy: 0.80

In [57]:
def build_model(hp):
  model = Sequential()
  units = hp.Int('units', min_value = 8,max_value = 128)
  model.add(Dense(units=units,activation='relu',input_dim=8))
  model.add(Dense(1,activation='sigmoid'))
  model.compile(optimizer='sgd',loss='binary_crossentropy',metrics=['accuracy'])
  return model

In [58]:
tuner = kt.RandomSearch(build_model,objective='val_accuracy',max_trials=5)

Reloading Tuner from ./untitled_project/tuner0.json


In [59]:
tuner.search(X_train,y_train,epochs=5,validation_data=(X_test,y_test))

In [60]:
tuner.get_best_hyperparameters()[0].values

{'units': 104}

In [61]:
model = tuner.get_best_models(num_models=1)[0]

In [56]:
model.fit(X_train,y_train,batch_size=32,epochs=100,initial_epoch=6,validation_data=(X_test,y_test))

Epoch 7/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 1s 15ms/step - accuracy: 0.7336 - loss: 0.5846 - val_accuracy: 0.7857 - val_loss: 0.5653
Epoch 8/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.7308 - loss: 0.5708 - val_accuracy: 0.7922 - val_loss: 0.5550
Epoch 9/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.7505 - loss: 0.5600 - val_accuracy: 0.7922 - val_loss: 0.5463
Epoch 10/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7531 - loss: 0.5575 - val_accuracy: 0.7922 - val_loss: 0.5383
Epoch 11/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7121 - loss: 0.5615 - val_accuracy: 0.7987 - val_loss: 0.5319
Epoch 12/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7336 - loss: 0.5526 - val_accuracy: 0.7987 - val_loss: 0.5261
Epoch 13/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.7408 - loss: 0.5503 - val_accuracy: 0.8052 - val_loss: 0.5208
Epoch 14/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7931 - loss: 0.5132 - val_accuracy: 0.80

In [106]:
def build_model(hp):
  model = Sequential()
  model.add(Dense(104,activation='relu',input_dim=8)) # First layer
  num_hidden_layers = hp.Int('num_hidden_layers', min_value = 1, max_value = 10)
  for i in range(num_hidden_layers):
    # Define a hyperparameter for the number of units in each hidden layer
    model.add(Dense(hp.Int(f'units_layer_{i}', min_value=8, max_value=128), activation='relu'))
    # Define a hyperparameter for the dropout rate
  dropout_rate = hp.Float('dropout_rate', min_value=0.0, max_value=0.5, step=0.1)
  model.add(Dense(1,activation='sigmoid'))
  # Define a hyperparameter for the optimizer
  optimizer=hp.Choice('optimizer',values = ['Adam','sgd','rmsprop','adadelta'])
  model.compile(optimizer=optimizer,loss='binary_crossentropy',metrics=['accuracy'])
  return model

In [107]:
tuner = kt.RandomSearch(
    build_model,
    objective='val_accuracy',
    max_trials=10,  # Increase max_trials for a better search
    executions_per_trial=2, # Run each trial twice to reduce variance
    directory='my_dir', # Specify a directory to save results
    project_name='diabetes_tuning_all'
)


Reloading Tuner from my_dir/diabetes_tuning_all/tuner0.json


In [108]:
tuner.search(X_train, y_train, epochs=10, validation_data=(X_test, y_test))

In [109]:
tuner.get_best_hyperparameters(num_trials=1)[0]

In [110]:
model = tuner.get_best_models(num_models=1)[0]

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
/usr/local/lib/python3.11/dist-packages/keras/src/saving/saving_lib.py:757: UserWarning: Skipping variable loading for optimizer 'rmsprop', because it has 2 variables whereas the saved optimizer has 26 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


In [96]:
def build_model(hp):
  model = Sequential()
  counter = 0
  for i in range(hp.Int('num_layers',min_value=1,max_value=10)):
    if counter == 0:
      model.add(Dense(
          hp.Int('units'+str(i),
                 min_value=8,max_value=128,step=8),
                      activation='relu',input_dim=8))
    else:
      model.add(Dense(
          hp.Int('units'+str(i),
                 min_value=8,max_value=128,step=8),
                      activation='relu'))
    counter += 1
  model.add(Dense(1,activation='sigmoid'))
  model.compile(optimizer=hp.Choice('optimizer',
                                    values = ['Adam','sgd','rmsprop','adadelta']),
                loss='binary_crossentropy',metrics=['accuracy'])
  return model

In [101]:
tuner = kt.RandomSearch(build_model,objective='val_accuracy',max_trials=5,
                        directory='my_dir_all_in_one', project_name='diabetes_all_in_one') # Added directory and project name for clarity


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [102]:
tuner.search(X_train,y_train,epochs=5,validation_data=(X_test,y_test))

Trial 5 Complete [00h 00m 03s]
val_accuracy: 0.6428571343421936

Best val_accuracy So Far: 0.7922077775001526
Total elapsed time: 00h 00m 19s


In [103]:
tuner.get_best_hyperparameters()[0].values

{'num_layers': 2, 'units0': 16, 'optimizer': 'Adam', 'units1': 8}

In [104]:
# Get the best hyperparameters (optional, but good for verification)
model = tuner.get_best_hyperparameters(num_trials=1)[0].values
print("Best hyperparameters found:")
print(model)

Best hyperparameters found:
{'num_layers': 2, 'units0': 16, 'optimizer': 'Adam', 'units1': 8}


In [ ]:
model